In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from IPython.display import Image, display

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from pathlib import Path

# Downloading images

In [ ]:
!mkdir orig

orig_img_urls = [
  'http://images.cocodataset.org/val2017/000000439715.jpg',
  'https://i.imgur.com/X6YxOhI.jpg',
  'https://i.imgur.com/qmx5irB.jpg',
  'https://i.imgur.com/JnQ1s0K.jpg',
  'https://i.imgur.com/Os9ximI.jpg',
  'https://i.imgur.com/K730ZTi.jpg',
  'https://i.imgur.com/MKomuMK.jpg',
  'https://i.imgur.com/hfmzzrm.jpg',
  'https://i.imgur.com/HSuOCP9.jpg',
  'https://i.imgur.com/TNzCyGH.jpg',
  'https://i.imgur.com/2vJ5GPL.jpg',
  'https://i.imgur.com/drIM7AM.jpg',
  'https://i.imgur.com/EsVdqKe.jpg',
  'https://i.imgur.com/pvwlVPn.jpg',
  'https://i.imgur.com/qkOR31z.jpg',
  'https://i.imgur.com/1wx8JlE.jpg',
  'https://i.imgur.com/FKROXfL.jpg',
  'https://i.imgur.com/V2exjLn.jpg',
  'https://i.imgur.com/KAgcikr.jpg',
  'https://i.imgur.com/XYUWXDE.jpg',
]

for i, img_url in enumerate(orig_img_urls):
  imgname = "{0:02d}.jpg".format(i)
  !wget $img_url -q -O orig/$imgname

# Variables

In [ ]:
THRESHOLD = 0.7
WIDTH = 700
HOME = Path.cwd()

# Object detection functions

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
# print(outputs["instances"].pred_classes)
# print(outputs["instances"].pred_boxes)

In [ ]:
def detect_boxes(config, folder_name):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file(config))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = THRESHOLD
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config)
  predictor = DefaultPredictor(cfg)

  orig_folder = HOME/'orig'
  folder_path = HOME/folder_name
  Path(folder_path).mkdir(parents=True, exist_ok=True)

  for file in list(Path(orig_folder).glob('*.jpg')):
    im = cv2.imread(str(file))
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(str(folder_path) + "/" + file.name, out.get_image()[:, :, ::-1])
    # cv2_imshow(out.get_image()[:, :, ::-1])
    

  # !zip -r -q /content/$folder_name\.zip /content/$folder_name

In [ ]:
def detect_panoptic(config, folder_name):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file(config))
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config)
  predictor = DefaultPredictor(cfg)


  orig_folder = HOME/'orig'
  folder_path = HOME/folder_name
  Path(folder_path).mkdir(parents=True, exist_ok=True)

  for file in list(Path(orig_folder).glob('*.jpg')):
    im = cv2.imread(str(file))
    panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
    cv2.imwrite(str(folder_path) + "/" + file.name, out.get_image()[:, :, ::-1])
    #cv2_imshow(out.get_image()[:, :, ::-1])

  # !zip -r -q /content/$folder_name\.zip /content/$folder_name

# Processing

In [ ]:
"""https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md"""

detect_boxes(
    'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml',
    'detection-faster-rcnn-R-50'
)

detect_boxes(
    'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml', 
    'detection-faster-rcnn-R-101'
)

detect_boxes(
    'COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', 
    'detection-faster-rcnn-X-101'
)

detect_boxes(
    'COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml',
    'keypoints-rcnn-X-101'
)

detect_boxes(
    'COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml',
    'segmentation-mask-rcnn-X-101'
)

detect_panoptic(
    'COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml', 
    'segmentation-panoptic-R-101'
)

# Visualization functions

In [ ]:
def visualize(folder, ):
  images = [
    Image(str(file), width=WIDTH) for file in list(folder.glob('*.jpg'))
  ]

  display(*images)

In [ ]:
folders = [
      x for x in Path(HOME).glob('**/*') if x.is_dir() 
      and not '.' in str(x.absolute()) 
      and not 'sample_data' in str(x.absolute())
]


for folder in folders:
  print(folder)

# Detection. Faster RCNN ResNet50

In [ ]:
visualize(Path(HOME / 'detection-faster-rcnn-R-50'))

# Detection. Faster RCNN ResNet101



In [ ]:
visualize(Path(HOME / 'detection-faster-rcnn-R-101'))

# Detection. Faster RCNN ResNeXt101



In [ ]:
visualize(Path(HOME / 'detection-faster-rcnn-X-101'))

# Keypoints. RCNN ResNeXt101



In [ ]:
visualize(Path(HOME / 'keypoints-rcnn-X-101'))

# Instance Segmentation. RCNN ResNeXt101



In [ ]:
visualize(Path(HOME / 'segmentation-mask-rcnn-X-101'))

# Panoptic Segmentation. ResNet101

In [ ]:
visualize(Path(HOME / 'segmentation-panoptic-R-101'))